In [1]:


import json
def prepare_label(dict_data)-> list:
    """
        dict_data : { e_id: {
                            'log_features':[] ,
                            'info_features':[] ,
                            'label' : 1 or 0
                            } 
                    }

    Returns:
        list_assemble_data : [[ *log_features , *info_features ],......]
        list_e_id : [ e_id_1 ,e_id_2,...... ]
        list_label : [ 0, 1, ..........]
    """    
    list_assemble_data = []
    list_e_id = []
    list_label = []
    for e_id,dict_ in dict_data.items():

        log_data  = dict_['features']

        label     = dict_['label']

        list_assemble_data.append(log_data)
        list_e_id.append(int(e_id))
        list_label.append(int(label))

    return list_assemble_data , list_e_id , list_label

dict_train=json.load(open('Final_Dataset\\transfor_matrix_train.json','r'))
dict_test = json.load(open('Final_Dataset\\transfor_matrix_test.json','r'))

list_data_train,list_e_id_train,list_label_train = prepare_label(dict_train)
list_data_test,list_e_id_test,list_label_test    = prepare_label(dict_test)


In [2]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
def predict_label_to_int(predict_label_list,threshold):

    predict_label_int = []
    for i in predict_label_list:
        value= i
        if value >threshold:label_ = int(1)
        else:label_ = int(0)
        predict_label_int.append(label_)

    return predict_label_int
def measure(predict_label_int,list_label_test):
    f1             = f1_score(predict_label_int,list_label_test)
   # accuracy = accuracy_score(predict_label_int,list_label_test)
    AUC =       roc_auc_score(predict_label_int,list_label_test)
    # precision = precision_score(predict_label_int,test_label.tolist())
    # recall = recall_score(predict_label_int,test_label.tolist())

    print(round(f1,4),round(AUC,4))

train_data =np.array( list_data_train)
test_data  =np.array( list_data_test)

Padding = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
train_data_padding = Padding.fit_transform(train_data)
test_data_padding  = Padding.fit_transform(test_data)



In [3]:

import xgboost as xgb
dtrain = xgb.DMatrix(train_data, list_label_train)
dtest = xgb.DMatrix(test_data,list_label_test)
num_rounds = 20
params = {}
watchlist = [(dtrain,'train'),(dtest,'test')]
XGB = xgb.train(
    params, 
    dtrain, 
    num_rounds,
    watchlist) 
XGB_predict_label = XGB.predict(dtest)

[0]	train-rmse:0.42097	test-rmse:0.43126
[1]	train-rmse:0.37544	test-rmse:0.39391
[2]	train-rmse:0.35010	test-rmse:0.37434
[3]	train-rmse:0.33624	test-rmse:0.36467
[4]	train-rmse:0.32858	test-rmse:0.35980
[5]	train-rmse:0.32439	test-rmse:0.35751
[6]	train-rmse:0.32190	test-rmse:0.35618
[7]	train-rmse:0.32027	test-rmse:0.35562
[8]	train-rmse:0.31931	test-rmse:0.35552
[9]	train-rmse:0.31835	test-rmse:0.35507
[10]	train-rmse:0.31760	test-rmse:0.35479
[11]	train-rmse:0.31705	test-rmse:0.35480
[12]	train-rmse:0.31653	test-rmse:0.35473
[13]	train-rmse:0.31593	test-rmse:0.35468
[14]	train-rmse:0.31539	test-rmse:0.35464
[15]	train-rmse:0.31502	test-rmse:0.35460
[16]	train-rmse:0.31461	test-rmse:0.35454
[17]	train-rmse:0.31411	test-rmse:0.35435
[18]	train-rmse:0.31378	test-rmse:0.35427
[19]	train-rmse:0.31346	test-rmse:0.35429


In [4]:
for i in [0.15,0.2,0.25,0.3,0.35]:
        
    XGB_predict_label_int = predict_label_to_int(XGB_predict_label,threshold=i)
    measure(
            XGB_predict_label_int,list_label_test)


0.8806 0.8384
0.8856 0.8364
0.8892 0.8304
0.892 0.8231
0.894 0.812


In [5]:
a = 1

In [6]:
for i in [0.35,0.4,0.45]:
        
    XGB_predict_label_int = predict_label_to_int(XGB_predict_label,threshold=i)
    measure(
            XGB_predict_label_int,list_label_test)


0.894 0.812
0.8956 0.8033
0.8954 0.7936
